# Limpieza del texto
En esta sección eliminaremos de nuestros documentos todo aquello que no sea relevante al significado del texto

In [19]:
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

Recuperamos y visualizamos algunos de nuestros datos

In [20]:
path_df = "dataframes/dirty_data.pickle"

with open(path_df, 'rb') as data:
    df = pickle.load(data)


In [21]:
df.head()

,file_name,content,category
0,001.txt,Algunos datos interesantes del cerebro\nBrenda...,Cerebro
1,002.txt,Hormonas definen diferencias de sexo a nivel c...,Cerebro
2,003.txt,EL CEREBRO ORGANIZACIÓN Y FUNCIÓN\nEl cerebro ...,Cerebro
3,004.txt,El Cerebro\nEs la parte más importante; está f...,Cerebro
4,005.txt,La nueva vision del cerebro\nLos científicos\n...,Cerebro


In [22]:
df.loc[0]['content']

'Algunos datos interesantes del cerebro\nBrenda Terrazas\n\nEl cerebro, elemento principal del Sistema Nervioso Central (SNC), se ubica dentro del cráneo, sumergido en el líquido cefalorraquídeo que lo protege, se estima tiene un peso promedio de entre 1.3 y 1.5 kilogramos, y se puede definir como un órgano complejo, quizá el más importante del cuerpo humano.\nEs responsable de la inteligencia, la interpretación de los sentidos, el control del comportamiento, el aprendizaje y el inicio de los movimientos corporales. También controla otras funciones en el cuerpo sin que tengamos que pensar en ello, como la respiración y el ritmo cardíaco.\nEl cerebro es impresionante y aunque los desarrollos tecnológicos modernos han ayudado a recolectar una enorme cantidad de datos, mucho permanece en el misterio. Por ello, te ofrecemos información interesante sobre el tema.\nParecido cerebral\nEl cerebro es resultado de la evolución de millones de años a partir de una “versión más simple”. Los nemátod

##### Eliminación de caracteres especiales

In [23]:
# \r, \n y ""
df["c1"] = df["content"].str.replace("\r"," ")
df["c1"] = df["c1"].str.replace("\n"," ")
df["c1"] = df["c1"].str.replace('"',"")

In [24]:
# pasando el texto a minusculas
df["c1"] = df["c1"].str.lower()

In [25]:
# eliminacion de signos de puntuacion
signos = list("?:!.,;")
df["c2"] = df["c1"]

for signo in signos:
    df["c2"] = df["c2"].str.replace(signo,"")

C:\Users\Bruno\AppData\Local\Temp\ipykernel_25672\1736598016.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["c2"] = df["c2"].str.replace(signo,"")


In [26]:
# eliminacion de pronombres posesivos
df["c3"] = df["c2"].str.replace("'s","")

##### Lematización
Proceso encargado de devolver las palabras derivadas a su forma base

In [27]:
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bruno\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Bruno\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Bruno\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [28]:
#lematizador
wordnetLem = WordNetLemmatizer()

In [29]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):

    # Create an empty list containing lemmatized words
    lemmatized_list = []

    # Save the text and its words into an object
    text = df.loc[row]['c2']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnetLem.lemmatize(word, pos="v"))

    # Join the list
    lemmatized_text = " ".join(lemmatized_list)

    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)


In [30]:
df['Content_Parsed_5'] = lemmatized_text_list


##### StopWords

In [31]:
# Downloading the stop words list
nltk.download('stopwords')
# Loading the stop words in english
stop_words = list(stopwords.words('english'))
stop_words.append('bbc')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bruno\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(
        regex_stopword, '')


C:\Users\Bruno\AppData\Local\Temp\ipykernel_25672\446418577.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(


In [33]:
df.loc[0]["Content_Parsed_6"]

'algunos datos interesantes del cerebro brenda terrazas  el cerebro elemento principal del sistema nervioso central (snc) se ubica dentro del cráneo sumergido en el líquido cefalorraquídeo que lo protege se estima tiene un peso promedio de entre 13  15 kilogramos  se puede definir como un órgano complejo quizá el más importante del cuerpo humano es responsable de la inteligencia la interpretación de los sentidos el control del comportamiento el aprendizaje  el inicio de los movimientos corporales también controla otras funciones en el cuerpo sin que tengamos que pensar en ello como la respiración  el ritmo cardíaco el cerebro es impresionante  aunque los desarrollos tecnológicos modernos han ayudado  recolectar una enorme cantidad de datos mucho permanece en el misterio por ello te ofrecemos información interesante sobre el tema parecido cerebral el cerebro es resultado de la evolución de millones de años  partir de una “versión más simple” los nemátodos moscas de la fruta pez cebra sa

Ahora podemos eliminar las columnas intermedias...

In [34]:
df= df[["file_name","category","Content_Parsed_6"]]
df = df.rename(columns={"Content_Parsed_6":"cleanContent"})

##### Obtención de TDIDF y transformación del dataset

In [42]:
ngram_range = (1, 2)
min_df = 10
max_df = 1.0
max_features = 300
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)

X_transformed = tfidf.fit_transform(df['cleanContent'])
X_transformed

#print(X_transformed)



<92x300 sparse matrix of type '<class 'numpy.float64'>'
	with 12793 stored elements in Compressed Sparse Row format>

In [43]:
tfidf.get_feature_names_out()

array(['10', '20', 'actividad', 'agua', 'al', 'algunos', 'alimentos',
       'ante', 'ante de', 'anterior', 'aos', 'aumento', 'aunque', 'base',
       'bien', 'cabeza', 'cada', 'campo', 'capacidad', 'caso', 'casos',
       'causa', 'cerebral', 'cerebro', 'ciencia', 'clulas', 'cmo', 'como',
       'como el', 'como la', 'con', 'con el', 'con la', 'con los',
       'con un', 'con una', 'conocimiento', 'consumo', 'contacto',
       'contra', 'control', 'cual', 'cualquier', 'cuando', 'cuando se',
       'cuenta', 'cuerpo', 'da', 'de', 'de este', 'de forma', 'de la',
       'de las', 'de los', 'de medicina', 'de que', 'de salud', 'de su',
       'de sus', 'de un', 'de una', 'debe', 'deben', 'debido', 'decir',
       'del', 'del cuerpo', 'dentro', 'desarrollo', 'desde', 'despus',
       'despus de', 'dolor', 'donde', 'durante', 'durante el', 'efecto',
       'efectos', 'ejemplo', 'ejercicio', 'el', 'el cuerpo',
       'el ejercicio', 'el sistema', 'el tratamiento', 'ella', 'embargo',
       '

In [44]:
# X_transformed
with open('dataframes/X_transformed.pickle', 'wb') as output:
    pickle.dump(X_transformed, output)